<a href="https://colab.research.google.com/github/ankit-rathi/Quantvesting_v2/blob/main/myStocks_Portfolio_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install pyxirr
!pip install ta
!pip install yfinance==0.2.59

  Installing build dependencies ... -

 \

 done


  Getting requirements to build wheel ... -

 done


...

 done
  Created wheel for ta: filename=ta-0.11.0-py3-none-any.whl size=29482 sha256=084bd49fb650bc71fa255f1d28eb888f4cf4ca81c0ee06996911369d030a8286
  Stored in directory: /home/runner/.cache/pip/wheels/ec/b6/81/d781ff040ee9432384560f29596ee14ca79d70529d4228e50f
Successfully built ta


  Attempting uninstall: protobuf
    Found existing installation: protobuf 6.33.1


    Uninstalling protobuf-6.33.1:
      Successfully uninstalled protobuf-6.33.1


  Attempting uninstall: yfinance
    Found existing installation: yfinance 0.2.66
    Uninstalling yfinance-0.2.66:
      Successfully uninstalled yfinance-0.2.66
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 0/2 [protobuf]

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2/2 [yfinance]


In [2]:
# import libraries

import numpy as np
import pandas as pd
import datetime
import warnings
warnings.filterwarnings('ignore')
import yfinance as yfin
import ta
import matplotlib.pyplot as plt
import requests

In [3]:
# notebook start time
import time
import datetime
import pytz

start_time = time.time()

# function to print date timestamp
def print_date_time():
  tz_NY = pytz.timezone('Asia/Kolkata')
  datetime_NY = datetime.datetime.now(tz_NY)
  print("Run date time (IST):", datetime_NY.strftime("%Y-%m-%d %H:%M:%S"))

In [4]:
# function to get booked and reserved amount
def get_amt():
  # fetch the JSON data from the URL
  url = "https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioAmts.json"
  response = requests.get(url)
  pf_amts = response.json()  # parse the JSON data

  # extract values from the JSON
  py_booked_amt_dm = pf_amts["py_booked_amt_dm"]
  py_booked_amt_sv = pf_amts["py_booked_amt_sv"]
  cy_booked_amt_dm = pf_amts["cy_booked_amt_dm"]
  cy_booked_amt_sv = pf_amts["cy_booked_amt_sv"]
  reserve_amt_dm = pf_amts["reserve_amt_dm"]
  reserve_amt_sv = pf_amts["reserve_amt_sv"]

  # perform calculations
  py_booked_amt = py_booked_amt_dm + py_booked_amt_sv
  cy_booked_amt = cy_booked_amt_dm + cy_booked_amt_sv
  reserve_amt = reserve_amt_dm + reserve_amt_sv
  total_booked_amt = py_booked_amt + cy_booked_amt

  return total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt

gen_output = 0

In [5]:
# function to format the amount
def format_amt(number):
    abs_number = abs(number)

    if abs_number >= 1_00_00_000:  # Crores
        formatted_amt = f"{number / 1_00_00_000:.2f} C"
    elif abs_number >= 1_00_000:  # Lakhs
        formatted_amt = f"{number / 1_00_000:.2f} L"
    elif abs_number >= 1_000:  # Thousands
        formatted_amt = f"{number / 1_000:.2f} K"
    else:
        formatted_amt = f"{number:.2f}"

    return formatted_amt

# function to set start and end date
def get_start_end_date():
  start_date = (datetime.date.today() + datetime.timedelta(days=-365)).strftime('%Y-%m-%d')
  end_date = (datetime.date.today() + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
  return start_date, end_date

# function to fetch my portfolio csv
def get_mypfs_df():
  mypfs_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myPortfolioStocks.csv')
  return mypfs_df

# function to fetch my prospects csv
def get_mypps_df():
  mypps_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myProspectsScrips.csv')
  return mypps_df

# function to fetch screener data
def get_myscreen_df():
  myscreen_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myScreenerDB.csv')
  return myscreen_df

# function to fetch momentum data
def get_myinvmt_df():
  myinvmt_df = pd.read_csv('https://raw.githubusercontent.com/ankit-rathi/Tradevesting_v1/main/data/myInvestments.csv')
  return myinvmt_df

# function to get the stock ids
def get_stock_ids(df_pf):
  stock_n100 = df_pf['Symbol'].unique()

  exclude = ['CADILAHC','MMTC', 'MASFIN']

  stock_ids = df_pf[~df_pf['Symbol'].isin(exclude) ]['Symbol'].unique()

  #mypf = mypf[mypf.Forecast.notnull()]
  #stock_ids = mypf['Symbol'].unique()

  stock_ids.sort()
  return stock_ids

# get features from screener data
def get_screener_features():
  myscreen_df = get_myscreen_df()
  cols = ['Symbol', 'EPS', 'MedPE', 'ROCE%', 'ROE%', 'CapType']
  return myscreen_df[cols]

# get relative strength
def get_relative_strength(stock_list):

    # Dictionary to store stock tickers and their corresponding percentage price change
    stock_changes = {}

    # Loop through each stock and fetch its price data
    for stock in stock_list:
        try:
            # Download the stock data for the given date range
            stock_data = yfin.Ticker(stock + '.NS').history(period='1mo', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

            # Calculate the percentage change for the stock
            if len(stock_data) > 0:
                start_price = stock_data['Close'].iloc[0]
                end_price = stock_data['Close'].iloc[-1]
                percent_change = round(((end_price - start_price) / start_price) * 100, 2)
                stock_changes[stock] = percent_change
            else:
                stock_changes[stock] = np.nan  # If no data is available, set to NaN

        except Exception as e:
            print(f"Error fetching data for {stock}: {e}")
            stock_changes[stock] = np.nan

    # Create a DataFrame with stock tickers and their percentage changes
    df = pd.DataFrame(list(stock_changes.items()), columns=['Symbol', 'Percent_Change'])

    # Drop any stocks with missing data (NaN values)
    df = df.dropna()

    # Calculate the percentile rank based on percentage change
    df['RSP'] = round(df['Percent_Change'].rank(pct=True) * 100, 2)
    cols = ['Symbol', 'RSP']
    df = df[cols]
    # Sort by percentile rank (optional)
    df = df.sort_values(by='RSP', ascending=False).reset_index(drop=True)

    return df


In [6]:
# function to get stock technicals
def stock_prec_dev(stock_symbol):
    #stock_symbol = 'ULTRACEMCO.NS'
    short_window = 20
    mid_window = 50
    long_window = 200
    moving_avg = 'SMA'

    start = datetime.datetime(*map(int, start_date.split('-')))
    end = datetime.datetime(*map(int, end_date.split('-')))

    stock_df = yfin.Ticker(stock_symbol + '.NS').history(period='max', interval='1d')[map(str.title, ['open', 'close', 'low', 'high', 'volume'])]

    stock_df['Max'] = round(max(stock_df['Close']),2)
    stock_df = stock_df[(stock_df.index <= end_date) & (stock_df.index >= start_date)]
    stock_df['200_SMA'] = round(stock_df['Close'].rolling(window = 200, min_periods = 1).mean(),0)
    stock_df['Dev%_200'] = round((stock_df['Close'] - stock_df['200_SMA'])*100/stock_df['200_SMA'],2)
    stock_df.dropna(axis = 0, inplace = True) # remove any null rows

    stock_df['20_SMA'] = stock_df['Close'].rolling(window=20).mean()
    stock_df['50_SMA'] = stock_df['Close'].rolling(window=50).mean()
    stock_df['Symbol'] = stock_symbol

    stock_df['Close'] = round(stock_df['Close'],2)
    stock_df['Min'] = round(min(stock_df['Close']),2)
    stock_df['RSI_14'] = ta.momentum.RSIIndicator(close=stock_df['Close'], window=14).rsi()
    stock_df['RSI_14'] = round(stock_df['RSI_14'],0)
    stock_df['Prev_Close'] = stock_df['Close'].shift(1)
    stock_df.drop(['Open', 'Low', 'High', 'Volume'], axis=1, inplace=True)
    stock_df = stock_df.tail(1)

    max_SMA = max(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    min_SMA = min(stock_df['20_SMA'].item(), stock_df['50_SMA'].item(), stock_df['200_SMA'].item())
    ABS_Spread = max_SMA - min_SMA
    stock_df['Spread%'] = round((ABS_Spread / stock_df['200_SMA'].item()) * 100,2)

    return stock_df

# function to compute stock attributes
def get_common_features(stock_ids, df_mypf):

  df_prec_dev = pd.DataFrame()
  df_tmp = get_screener_features()
  df_rs = get_relative_strength(stock_ids)

  for stock_id in stock_ids:
      tmp = stock_prec_dev(stock_id)
      tmp = tmp.reset_index()
      df_prec_dev = pd.concat([df_prec_dev, tmp], ignore_index = True)
  df_prec_dev = pd.merge(df_prec_dev, df_mypf, on= 'Symbol')
  df_prec_dev = pd.merge(df_prec_dev, df_tmp, on= 'Symbol', how='left')
  df_prec_dev = pd.merge(df_prec_dev, df_rs, on= 'Symbol', how='left')
  #print(df_prec_dev.columns)
  df_prec_dev['Curr_PE'] = round(df_prec_dev['Close']/df_prec_dev['EPS'],1)
  df_prec_dev['Dev%_PE'] = round((df_prec_dev['Curr_PE'] - df_prec_dev['MedPE'])*100/df_prec_dev['MedPE'],2)
  df_prec_dev['Conviction'] = df_prec_dev['Conviction'] + '-' + df_prec_dev['CapType']
  return df_prec_dev

# function to arrange stock features
def arrange_features(df_stocks, common_cols, diff_cols):
  df_stocks_common = df_stocks[common_cols].drop_duplicates()
  df_stocks_diff = df_stocks[diff_cols]
  df_stocks_diff['Investment'] = df_stocks_diff['AvgCost'] * df_stocks_diff['Shares']
  df_stocks_diff = df_stocks_diff.groupby(['Symbol'])[['Shares', 'Investment']].aggregate(['sum']).reset_index()
  df_stocks_diff.columns = ['Symbol', 'Shares', 'Investment']
  df_stocks_diff['AvgCost'] = round(df_stocks_diff['Investment']/df_stocks_diff['Shares'],2)
  df_stocks = pd.merge(df_stocks_diff, df_stocks_common, on='Symbol')
  return df_stocks

# plot fact distribution across dimension
def plot_pie_chart(df, dimension, fact):
  # grouping the data by category and calculating the sum of fact for each type
  grouped_data = df.groupby(dimension)[fact].sum()

  # sorting the grouped data in descending order
  grouped_data = grouped_data.sort_values(ascending=False)

  # creating a pie chart
  grouped_data.plot.pie(autopct='%1.1f%%', startangle=90, figsize=(6, 6))

  # adding a title and displaying the plot
  plt.title(f'{dimension} {fact} Distribution')
  plt.ylabel('')  # To hide the y-label
  plt.show()

In [7]:
# function to get portfolio features

def get_portfolio_features(df_common_features):

  print_date_time()
  print('-------------------')

  #df_common_features["Target"] = df_common_features["Target"].fillna(df_common_features["Max"])
  df_common_features['NTT'] = np.where(df_common_features['Strategy']== 'NTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['LTT'] = np.where(df_common_features['Strategy']== 'BTT', df_common_features["Target"], df_common_features['Max'])
  df_common_features['BOL'] = df_common_features['Min']

  tmp_df = df_common_features[df_common_features['Symbol'].isin(mypf_df[mypf_df['InPortfolio'] != 'NA'].Symbol.values)]
  print('qualified stocks: '+str(len(tmp_df['Symbol'].unique())))
  tmp_df1 = tmp_df[tmp_df['LatestQtr'] == 1]
  print('with latest results: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df1 = tmp_df1[tmp_df1['StarStock'] == 1]
  print('still star stocks: '+str(len(tmp_df1['Symbol'].unique())))
  tmp_df['Investment'] = tmp_df['AvgCost'] * tmp_df['Shares']
  tmp_df['Current'] = round(tmp_df['Close'] * tmp_df['Shares'],0)
  tmp_df['Previous'] = tmp_df['Prev_Close'] * tmp_df['Shares']
  tmp_df['EstimatedST'] = tmp_df['NTT'] * tmp_df['Shares']
  tmp_df['EstimatedLT'] = tmp_df['LTT'] * tmp_df['Shares']
  tmp_df['Current P/L'] = round((tmp_df['Current'] - tmp_df['Investment']),0)
  tmp_df['Today P/L%'] = round((tmp_df['Current'] - tmp_df['Previous'])*100/tmp_df['Previous'],2)
  tmp_df['Current P/L%'] = round((tmp_df['Current'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedST P/L%'] = round((tmp_df['EstimatedST'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['EstimatedLT P/L%'] = round((tmp_df['EstimatedLT'] - tmp_df['Investment'])*100/tmp_df['Investment'],2)
  tmp_df['NTT%'] = round((tmp_df['NTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['LTT%'] = round((tmp_df['LTT'] - tmp_df['Close'])*100/tmp_df['Close'],2)
  tmp_df['Gained%'] = round((tmp_df['Close'] - tmp_df['BOL'])*100/tmp_df['BOL'],2)
  investment = round(sum(tmp_df['AvgCost']*tmp_df['Shares']),0)
  current = round(sum(tmp_df['Close']*tmp_df['Shares']),0)
  tmp_df['InitAlloc%'] = round(tmp_df['Investment']*100/investment,2)
  tmp_df['CurrAlloc%'] = round(tmp_df['Current']*100/current,2)
  tmp_df['FTT'] = tmp_df['LTT']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT'] = tmp_df['NTT']
  tmp_df['FTT%'] = tmp_df['LTT%']
  tmp_df.loc[tmp_df['Strategy'] == 'NTT', 'FTT%'] = tmp_df['NTT%']
  tmp_df['FTT Amt'] = round(tmp_df['FTT%'] * tmp_df['Current']/100,0)
  tmp_df['OTT%'] = round((tmp_df['FTT'] - tmp_df['AvgCost'])*100/tmp_df['AvgCost'],2)
  tmp_df['RRR Ind'] = round(tmp_df['Current P/L']/tmp_df['FTT Amt'],2)
  tmp_df['Risk Ind'] = round(tmp_df['Current P/L%']*tmp_df['CurrAlloc%'],0)

  return tmp_df

# function to print portfolio features
def print_portfolio_stats(df_portfolio_features, myinvmt_df):
  from pyxirr import xirr

  total_booked_amt, reserve_amt, cy_booked_amt, py_booked_amt = get_amt()

  dates = myinvmt_df['Date'].values
  dates = np.append(dates, datetime.date.today().strftime('%d-%b-%y'))
  investment = myinvmt_df['Investment'].values
  dates= pd.to_datetime(dates)

  current = round(sum(df_portfolio_features['Close']*df_portfolio_features['Shares']),0) + reserve_amt
  investment_xirr = np.append(investment, current)
  cagr = round(xirr(pd.DataFrame({"dates": dates, "amounts": investment_xirr}))*100,2)

  investment = -sum(investment)
  invested = round(sum(df_portfolio_features['AvgCost']*df_portfolio_features['Shares']),0) + reserve_amt
  previous = round(sum(df_portfolio_features['Prev_Close']*df_portfolio_features['Shares']),0) + reserve_amt
  cy_invested = investment + py_booked_amt

  today_pnl_amount = current-previous
  today_pnl_percentage = round((current-previous)*100/previous,2)

  curr_pnl_amount = current-invested
  curr_pnl_percentage = round((curr_pnl_amount)*100/(cy_invested),2)

  cy_pnl_amount = cy_booked_amt + curr_pnl_amount
  cy_pnl_percentage = round((cy_pnl_amount)*100/cy_invested,2)

  overall_pnl_amount = total_booked_amt + curr_pnl_amount
  overall_pnl_percentage = round((overall_pnl_amount)*100/investment,2)

  estimate_st = round(sum(df_portfolio_features['FTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_st_pnl_amount = estimate_st-current
  est_st_pnl_percentage = round((est_st_pnl_amount)*100/current,2)

  estimate_lt = round(sum(df_portfolio_features['LTT']*df_portfolio_features['Shares']),0)  + reserve_amt
  est_lt_pnl_amount = estimate_lt-current
  est_lt_pnl_percentage = round((est_lt_pnl_amount)*100/current,2)

  total_profit = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] > 0]['Investment']),0)
  total_loss = round(sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Current']) - sum(df_portfolio_features[df_portfolio_features['Current P/L%'] < 0]['Investment']),0)

  cy_booked_percentage = round((cy_booked_amt)*100/current,2)
  py_booked_percentage = round((py_booked_amt)*100/investment,2)
  total_booked_percentage = round((total_booked_amt)*100/investment,2)

  print('-------------------')
  print('Initial Investment: ', format_amt(investment))
  print('CY Investment: ', format_amt(cy_invested))
  print('Reserve: ', format_amt(reserve_amt))
  print('Current: ',  format_amt(current))
  print('-------------------')
  print('Today PnL: '+ '{} ({}%)'.format(format_amt(today_pnl_amount), today_pnl_percentage))
  print('Current PnL: '+ '{} ({}%)'.format(format_amt(curr_pnl_amount), curr_pnl_percentage))
  print('CY Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(cy_pnl_amount), cy_pnl_percentage))
  print('-------------------')
  print('Total profit: ', format_amt(total_profit))
  print('Total loss: ', format_amt(total_loss))
  print('-------------------')
  print('Total Booked + Current PnL: '+ '{} ({}%)'.format(format_amt(overall_pnl_amount), overall_pnl_percentage))
  print('Total Booked PnL: '+ '{} ({}%)'.format(format_amt(total_booked_amt), total_booked_percentage))
  print('Curr Year Booked PnL: '+ '{} ({}%)'.format(format_amt(cy_booked_amt), cy_booked_percentage))
  print('Prev Year Booked PnL: '+ '{} ({}%)'.format(format_amt(py_booked_amt), py_booked_percentage))
  print('===================')
  print('Est FTT: ',  format_amt(estimate_st))
  print('Est FTT PnL: '+ '{} ({}%)'.format(format_amt(est_st_pnl_amount), est_st_pnl_percentage))

  print('===================')
  print('Deployed: ', format_amt(investment))

  print('Current: ', format_amt(current))

  print('CAGR/XIRR %: '+'{}%'.format(cagr))

In [8]:
# get start and end date
start_date, end_date = get_start_end_date()

# get portfolio and prospects data
mypfs_df = get_mypfs_df()
mypps_df = get_mypps_df()
myinvmt_df = get_myinvmt_df()

# merge above datasets
mypf_df = pd.merge(mypfs_df, mypps_df, on="Symbol")

# seggregate dm and sv portfolio
dm_pf = mypf_df[mypf_df['InPortfolio'] == 'DM']
sv_pf = mypf_df[mypf_df['InPortfolio'] == 'SV']

# build portfolio stock dataframe
dm_stocks = get_stock_ids(dm_pf)
sv_stocks = get_stock_ids(sv_pf)
df_stocks = pd.concat([dm_pf,sv_pf], ignore_index = True)

# arrange common and diff stock features
common_cols = ['Symbol', 'Target', 'Criteria', 'Strategy', 'CumlRnk', 'LatestQtr', 'StarStock', 'Conviction', 'Category', 'InFolio', 'XIRR', 'MBQ']
diff_cols = ['Symbol', 'AvgCost', 'Shares']
df_stocks = arrange_features(df_stocks, common_cols, diff_cols)

# get common features
stock_ids = df_stocks['Symbol'].values
df_common_features = get_common_features(stock_ids, df_stocks)

df_common_features.reset_index(drop=True, inplace=True)
df_common_features.drop(['Date'], axis=1, inplace=True)
# get and print portfolio features
df_portfolio_features = get_portfolio_features(df_common_features)

print_portfolio_stats(df_portfolio_features, myinvmt_df)

df = df_portfolio_features
#plot_pie_chart(df, 'CapType', 'Current')
list_ox40n = list(df[df['MBQ'].str.contains('OX40N', na=False)]['Symbol'].values)

Run date time (IST): 2025-11-26 15:06:12
-------------------
qualified stocks: 87
with latest results: 25
still star stocks: 15
-------------------
Initial Investment:  1.26 C
CY Investment:  1.53 C
Reserve:  400.00
Current:  1.44 C
-------------------
Today PnL: 1.47 L (1.04%)
Current PnL: -22.14 L (-14.51%)
CY Booked + Current PnL: -8.25 L (-5.4%)
-------------------
Total profit:  1.63 L
Total loss:  -23.77 L
-------------------
Total Booked + Current PnL: 18.73 L (14.91%)
Total Booked PnL: 40.87 L (32.53%)
Curr Year Booked PnL: 13.89 L (9.67%)
Prev Year Booked PnL: 26.98 L (21.48%)
Est FTT:  2.31 C
Est FTT PnL: 87.55 L (60.94%)
Deployed:  1.26 C
Current:  1.44 C
CAGR/XIRR %: 7.63%


In [9]:
# top 5 near their targets
cols = ['Symbol', 'FTT','Today P/L%', 'Current P/L%', 'FTT%', 'OTT%', 'FTT Amt', 'Current P/L', 'Current', 'Dev%_PE', 'RSI_14','Conviction', 'Spread%',  'CumlRnk', 'RRR Ind', 'CurrAlloc%', 'Gained%', 'Criteria', 'Strategy', 'Category']
df_tmp = df_portfolio_features[~df_portfolio_features['Symbol'].isin(['ENRIN','BLUSPRING','DIGITIDE'])]
df_tmp = df_tmp.sort_values(by = 'FTT Amt', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.28,-7.26,7.84,0.02,12692.0,-12665.0,161894.0,146.65,73.0,M-SC,7.91,234.0,-1.00,1.13,40.82,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.08,16.49,0.09,14279.0,-14186.0,86591.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.76,OX40N,NTT,DURABLES
83,VOLTAS,1530.00,2.33,8.58,10.24,19.69,21318.0,16443.0,208185.0,-2.04,53.0,X-MC,3.40,78.0,0.77,1.45,15.96,XY25,NTT,AC
35,ICICIGI,2252.93,0.55,7.23,12.09,20.19,22600.0,12610.0,186930.0,-16.02,52.0,X-MC,7.47,68.0,0.56,1.30,23.44,X40,ATH,INSURANCE
53,NESTLEIND,1377.00,1.21,12.71,7.70,21.40,23036.0,33743.0,299169.0,4.55,59.0,X-LC,7.80,12.0,1.46,2.08,20.20,XY25,NTT,FMCG


In [10]:
# top 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
66,SIEMENS,4671.50,4.39,-10.78,40.68,25.51,67543.0,-20060.0,166035.0,1.98,67.0,H-LC,1.89,49.0,-0.30,1.16,22.35,AR,ATH,ELECTRICAL
82,VBL,671.64,3.72,-5.92,44.21,35.67,131369.0,-18694.0,297148.0,-17.17,54.0,X-LC,5.97,4.0,-0.14,2.07,7.12,X40N,ATH,FMCG
64,SFL,1287.00,3.52,-39.59,109.61,26.63,173635.0,-103806.0,158412.0,20.06,36.0,M-SC,4.95,239.0,-0.60,1.10,3.52,XY24,NTT,MISC
0,5PAISA,593.00,3.00,-34.59,72.61,12.90,103772.0,-75579.0,142917.0,110.34,59.0,H-SC,13.97,161.0,-0.73,0.99,18.83,OX40N,NTT,FINANCE
7,ATULAUTO,844.00,2.95,-20.22,82.47,45.57,129698.0,-39865.0,157267.0,3402.70,46.0,M-SC,4.13,244.0,-0.31,1.09,11.50,XY24,NTT,AUTO


In [11]:
# bottom 5 today
df_tmp = df_portfolio_features.sort_values(by = 'Today P/L%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
44,JCHAC,2282.00,-1.84,-24.90,33.14,-0.01,31809.0,-31821.0,95984.0,19552.17,31.0,M-SC,1.60,233.0,-1.00,0.67,8.69,OX40N,NTT,AC
8,AWL,485.00,-1.66,-15.51,81.68,53.50,208233.0,-46814.0,254937.0,-59.16,44.0,X-MC,4.06,58.0,-0.22,1.77,13.23,XY24,NTT,FMCG
56,QUESS,424.00,-1.59,-27.54,97.14,42.84,45756.0,-17903.0,47103.0,-52.62,42.0,X-SC,37.46,83.0,-0.39,0.33,2.49,XY24,NTT,MISC
69,STARHEALTH,761.00,-1.36,-5.28,55.46,47.26,147683.0,-14835.0,266288.0,29.34,48.0,H-SC,15.48,171.0,-0.10,1.85,42.67,XY24,NTT,INSURANCE
38,INDIAMART,4810.62,-0.95,0.43,105.85,106.72,131107.0,525.0,123861.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.86,23.90,AR,ATH,MISC


In [12]:
# top 5 to exit based on CumlRnk
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > -1) & (df_tmp['Current P/L%'] < 1)].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
38,INDIAMART,4810.62,-0.95,0.43,105.85,106.72,131107.0,525.0,123861.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.86,23.90,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.24,-0.88,36.83,35.63,76511.0,-1834.0,207740.0,-14.97,42.0,H-MC,4.17,119.0,-0.02,1.45,15.65,AR,ATH,PHARMA


In [13]:
# OX40N stocks to exit
df_tmp = df_portfolio_features[(df_portfolio_features['Symbol'].isin(list_ox40n))].sort_values(by = 'Current P/L%', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
25,FINCABLES,1641.55,0.17,-4.41,117.51,107.91,165841.0,-6515.0,141129.0,-22.17,37.0,M-SC,10.48,216.0,-0.04,0.98,0.92,OX40N,ATH,CABLES
39,INDIGOPNTS,1408.00,2.28,-7.26,7.84,0.02,12692.0,-12665.0,161894.0,146.65,73.0,M-SC,7.91,234.0,-1.00,1.13,40.82,OX40N,NTT,PAINTS
78,TTKPRESTIG,770.00,0.03,-14.08,16.49,0.09,14279.0,-14186.0,86591.0,101.07,50.0,M-SC,3.70,253.0,-0.99,0.60,13.76,OX40N,NTT,DURABLES
47,KANSAINER,340.00,1.16,-22.00,45.49,13.47,95679.0,-59337.0,210330.0,-67.92,32.0,H-SC,1.28,158.0,-0.62,1.46,6.95,XY24,NTT,PAINTS
67,SIS,528.00,0.92,-23.17,57.82,21.26,49326.0,-25722.0,85310.0,2013.99,51.0,H-SC,3.86,166.0,-0.52,0.59,15.26,OX40N,NTT,MISC


In [14]:
# top 5 to book for rotation
df_tmp = df_portfolio_features[(df_portfolio_features['Current P/L%'] > 20) ].sort_values(by = 'CumlRnk', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category


In [15]:
# top 5 to monitor for exit
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['Current P/L%'] > 0) ].sort_values(by = 'Dev%_PE', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.92,2.69,67.76,72.27,117902.0,4552.0,174000.0,-11.98,61.0,M-LC,4.39,99.0,0.04,1.21,9.03,XR,NTT,IT
38,INDIAMART,4810.62,-0.95,0.43,105.85,106.72,131107.0,525.0,123861.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.86,23.90,AR,ATH,MISC


In [16]:
# top 5 to book for rotation from weak conviction
df_tmp = df_portfolio_features[~df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[~df_tmp['Criteria'].isin(['XY25','XY24','X40', 'X40N'])].sort_values(by = 'RRR Ind', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
85,WIPRO,420.00,1.92,2.69,67.76,72.27,117902.0,4552.0,174000.0,-11.98,61.0,M-LC,4.39,99.0,0.04,1.21,9.03,XR,NTT,IT
38,INDIAMART,4810.62,-0.95,0.43,105.85,106.72,131107.0,525.0,123861.0,-50.25,39.0,H-SC,3.40,139.0,0.00,0.86,23.90,AR,ATH,MISC
86,ZYDUSLIFE,1286.17,1.24,-0.88,36.83,35.63,76511.0,-1834.0,207740.0,-14.97,42.0,H-MC,4.17,119.0,-0.02,1.45,15.65,AR,ATH,PHARMA
25,FINCABLES,1641.55,0.17,-4.41,117.51,107.91,165841.0,-6515.0,141129.0,-22.17,37.0,M-SC,10.48,216.0,-0.04,0.98,0.92,OX40N,ATH,CABLES
84,WHIRLPOOL,2270.00,0.02,-4.19,89.88,81.92,98855.0,-4812.0,109986.0,-43.90,36.0,M-SC,3.80,236.0,-0.05,0.77,30.21,XR,NTT,DURABLES


In [17]:
# top 5 to accumulate based on lowest RSI_14
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'RSI_14', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.61,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
46,JSWINFRA,342.00,0.95,-9.08,26.74,15.23,48707.0,-18198.0,182149.0,-22.32,28.0,X-MC,6.78,66.0,-0.37,1.27,20.24,X40N,NTT,REALTY
76,TMPV,600.00,1.89,-14.91,67.08,42.18,157780.0,-41199.0,235211.0,-23.08,29.0,X-LC,1.06,3.0,-0.26,1.64,1.89,XY24,NTT,AUTO
9,BAJAJHFL,181.50,0.14,-14.03,72.68,48.45,125286.0,-28126.0,172380.0,-20.30,30.0,X-MC,7.08,64.0,-0.22,1.20,0.14,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.39,-4.59,40.19,33.76,77008.0,-9210.0,191610.0,-32.64,33.0,X-MC,3.54,57.0,-0.12,1.33,0.37,X40,ATH,FMCG


In [18]:
# top 5 to accumulate based on Spread and CumlRnk
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])]
df_tmp = df_tmp[(df_tmp['CumlRnk'] < 100)].sort_values(by = 'Spread%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
43,ITC,452.0,0.42,-1.66,12.30,10.44,24209.0,-3316.0,196822.0,-41.15,43.0,X-LC,0.89,5.0,-0.14,1.37,3.81,X40,NTT,FMCG
76,TMPV,600.0,1.89,-14.91,67.08,42.18,157780.0,-41199.0,235211.0,-23.08,29.0,X-LC,1.06,3.0,-0.26,1.64,1.89,XY24,NTT,AUTO
3,ACC,3906.0,0.72,-20.89,107.49,64.15,202350.0,-49701.0,188250.0,-53.69,56.0,X-SC,1.58,82.0,-0.25,1.31,5.60,XY24,BTT,CEMENT
36,ICICIPRULI,790.0,1.40,2.92,27.26,30.97,49584.0,5156.0,181894.0,-21.26,56.0,X-MC,1.69,75.0,0.10,1.27,15.84,X40,ATH,INSURANCE
66,SIEMENS,4671.5,4.39,-10.78,40.68,25.51,67543.0,-20060.0,166035.0,1.98,67.0,H-LC,1.89,49.0,-0.30,1.16,22.35,AR,ATH,ELECTRICAL


In [19]:
# top 5 to accumulate based on lowest Gained%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'Gained%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
11,BATAINDIA,2096.00,-0.61,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
9,BAJAJHFL,181.50,0.14,-14.03,72.68,48.45,125286.0,-28126.0,172380.0,-20.30,30.0,X-MC,7.08,64.0,-0.22,1.20,0.14,X40N,ATH,FINANCE
55,PGHH,17907.65,-0.39,-4.59,40.19,33.76,77008.0,-9210.0,191610.0,-32.64,33.0,X-MC,3.54,57.0,-0.12,1.33,0.37,X40,ATH,FMCG
54,PAGEIND,51605.08,1.19,-4.74,32.34,26.07,50444.0,-7760.0,155980.0,-29.81,39.0,X-MC,9.18,55.0,-0.15,1.09,1.19,X40,ATH,APPARELS
76,TMPV,600.00,1.89,-14.91,67.08,42.18,157780.0,-41199.0,235211.0,-23.08,29.0,X-LC,1.06,3.0,-0.26,1.64,1.89,XY24,NTT,AUTO


In [20]:
# top 5 to accumulate based on lowest CurrAlloc%
df_tmp = df_portfolio_features[df_portfolio_features['Conviction'].isin(['X-LC','H-LC','X-MC','X-SC'])].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
56,QUESS,424.00,-1.59,-27.54,97.14,42.84,45756.0,-17903.0,47103.0,-52.62,42.0,X-SC,37.46,83.0,-0.39,0.33,2.49,XY24,NTT,MISC
60,RELAXO,1176.00,2.57,-46.84,186.51,52.31,143920.0,-67995.0,77165.0,-43.37,37.0,X-SC,4.10,91.0,-0.47,0.54,3.29,X40N,NTT,FOOTWEAR
11,BATAINDIA,2096.00,-0.61,-35.48,109.56,35.20,90953.0,-45653.0,83017.0,8.88,24.0,X-SC,13.04,92.0,-0.50,0.58,0.00,X40,NTT,FOOTWEAR
22,DIXON,18931.72,2.93,-4.80,27.72,21.58,36980.0,-6732.0,133407.0,-52.01,38.0,X-MC,6.70,56.0,-0.18,0.93,18.11,X40N,ATH,IT
52,MEDANTA,1486.00,1.00,5.38,17.74,24.08,23733.0,6833.0,133783.0,-3.08,50.0,X-SC,5.18,89.0,0.29,0.93,26.05,XY24,NTT,HEALTHCARE


In [21]:
# top 5 to accumulate based on CumlRnk
df_tmp = df_portfolio_features.sort_values(by = 'CumlRnk', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
74,TCS,4389.96,1.39,-14.07,38.82,19.28,115399.0,-48690.0,297266.0,-25.24,63.0,X-LC,7.35,1.0,-0.42,2.07,9.89,X40,ATH,IT
41,INFY,2275.00,1.71,5.97,46.14,54.87,152990.0,18689.0,331577.0,-17.44,61.0,X-LC,2.85,2.0,0.12,2.31,14.75,X40,BTT,IT
76,TMPV,600.00,1.89,-14.91,67.08,42.18,157780.0,-41199.0,235211.0,-23.08,29.0,X-LC,1.06,3.0,-0.26,1.64,1.89,XY24,NTT,AUTO
82,VBL,671.64,3.72,-5.92,44.21,35.67,131369.0,-18694.0,297148.0,-17.17,54.0,X-LC,5.97,4.0,-0.14,2.07,7.12,X40N,ATH,FMCG
43,ITC,452.00,0.42,-1.66,12.30,10.44,24209.0,-3316.0,196822.0,-41.15,43.0,X-LC,0.89,5.0,-0.14,1.37,3.81,X40,NTT,FMCG


In [22]:
# top 5 for average up
df_tmp = df_portfolio_features[(df_portfolio_features['Dev%_200'] > 5)].sort_values(by = 'CurrAlloc%', ascending=True)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
6,ASIANTILES,137.00,0.33,-12.43,107.45,81.67,88274.0,-11656.0,82154.0,7388.89,53.0,L-SC,19.03,271.0,-0.13,0.57,59.09,XR,NTT,CERAMICS
68,SONACOMS,806.63,0.84,-12.55,59.57,39.54,52697.0,-12697.0,88462.0,-31.56,65.0,M-SC,6.63,220.0,-0.24,0.62,24.74,AR,ATH,AUTO
51,MASFIN,398.61,0.13,-5.68,29.40,22.05,27170.0,-5565.0,92415.0,-18.70,48.0,H-SC,7.38,164.0,-0.20,0.64,34.33,XR,ATH,FINANCE
70,SURYODAY,216.00,1.71,-18.18,50.38,23.03,73813.0,-32558.0,146513.0,60.09,52.0,H-SC,10.19,167.0,-0.44,1.02,45.37,XR,NTT,BANKS
39,INDIGOPNTS,1408.00,2.28,-7.26,7.84,0.02,12692.0,-12665.0,161894.0,146.65,73.0,M-SC,7.91,234.0,-1.00,1.13,40.82,OX40N,NTT,PAINTS


In [23]:
# top 5 RSP
df_tmp = df_portfolio_features.sort_values(by = 'RSP', ascending=False)
df_tmp[cols].head()

,Symbol,FTT,Today P/L%,Current P/L%,FTT%,OTT%,FTT Amt,Current P/L,Current,Dev%_PE,RSI_14,Conviction,Spread%,CumlRnk,RRR Ind,CurrAlloc%,Gained%,Criteria,Strategy,Category
39,INDIGOPNTS,1408.00,2.28,-7.26,7.84,0.02,12692.0,-12665.0,161894.0,146.65,73.0,M-SC,7.91,234.0,-1.00,1.13,40.82,OX40N,NTT,PAINTS
5,ASIANPAINT,3460.25,-0.18,0.34,20.54,20.95,51889.0,854.0,252622.0,-0.63,75.0,X-LC,13.93,31.0,0.02,1.76,36.10,X40,ATH,PAINTS
40,INDUSINDBK,1800.00,1.34,-37.16,111.47,32.89,54083.0,-28688.0,48518.0,-699.18,63.0,L-MC,5.58,259.0,-0.53,0.34,33.62,XR,NTT,BANKS
0,5PAISA,593.00,3.00,-34.59,72.61,12.90,103772.0,-75579.0,142917.0,110.34,59.0,H-SC,13.97,161.0,-0.73,0.99,18.83,OX40N,NTT,FINANCE
79,UNITDSPR,1644.00,2.09,7.27,12.58,20.77,31597.0,17028.0,251172.0,-2.02,66.0,X-MC,3.73,62.0,0.54,1.75,14.69,X40N,NTT,BREWERIES


In [24]:
# Top N allocation
df_tmp = df_portfolio_features.sort_values(by = 'CurrAlloc%', ascending=False)
top_n_values = [10, 25, 50]

sum_df = pd.DataFrame({
    'Top_N': top_n_values,
    'Sum_Alloc%': [df_tmp['CurrAlloc%'].head(n).sum() for n in top_n_values]
})

sum_df

,Top_N,Sum_Alloc%
0,10,19.76
1,25,44.58
2,50,75.49


In [25]:
# market-cap-wise allocation
df_tmp = df_portfolio_features[cols]
df_tmp.groupby(df_tmp['Conviction'].str[-2:])['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
SC    45.60
MC    29.24
LC    25.17
Name: CurrAlloc%, dtype: float64

In [26]:
# criteria-wise allocation
df_portfolio_features.groupby('Criteria')['CurrAlloc%'].sum().sort_values(ascending=False)

Criteria
XY24     27.84
X40      21.15
X40N     11.96
XR        9.63
AR        9.08
XY25      9.07
OX40N     7.68
X200      1.84
SR        0.98
MH        0.78
Name: CurrAlloc%, dtype: float64

In [27]:
# conviction-wise allocation
df_portfolio_features.groupby('Conviction')['CurrAlloc%'].sum().sort_values(ascending=False)

Conviction
H-SC    23.88
X-MC    22.79
X-LC    19.99
M-SC    12.06
X-SC     8.26
H-MC     4.75
H-LC     3.00
L-SC     1.40
M-MC     1.36
M-LC     1.21
L-LC     0.97
L-MC     0.34
Name: CurrAlloc%, dtype: float64

In [28]:
# sector-wise stats
df_tmp = df_portfolio_features.groupby('Category')[['CurrAlloc%', 'Current', 'Current P/L', 'FTT Amt']].sum().sort_values(by=['Current', 'Current P/L'], ascending=False)
df_tmp['Current P/L%'] = round(df_tmp['Current P/L'] * 100 / df_tmp['Current'], 2)
df_tmp['FTT%'] = round(df_tmp['FTT Amt'] * 100 / df_tmp['Current'], 2)
cols = ['CurrAlloc%', 'Current P/L%', 'FTT%']
df_tmp[cols].sort_values(by=['CurrAlloc%'], ascending=False)

,CurrAlloc%,Current P/L%,FTT%
Category,,,
FMCG,16.73,-4.71,38.57
IT,13.76,-14.48,73.57
FINANCE,9.40,-14.54,62.94
MISC,7.43,-21.77,71.66
ELECTRICAL,6.00,-8.95,48.85
PAINTS,5.97,-7.65,23.64
INSURANCE,4.42,0.46,34.62
PHARMA,3.89,-0.26,32.35
AUTO,3.35,-19.50,70.73


In [29]:
# money to be made criteria-wise
df_portfolio_features.groupby('Criteria')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Criteria,,
XY24,3032150.0,21
XR,1281160.0,13
AR,1269677.0,10
X40,1002521.0,14
X40N,732588.0,9
OX40N,687534.0,10
XY25,337347.0,6
SR,280153.0,2
MH,72485.0,1


In [30]:
# money to be made conviction-wise
df_portfolio_features.groupby('Conviction')['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

,sum,count
Conviction,,
H-SC,3546668.0,25
M-SC,1326084.0,15
X-MC,1210270.0,16
X-LC,892218.0,11
X-SC,734779.0,8
H-MC,396635.0,3
L-SC,260368.0,3
H-LC,126768.0,2
M-LC,117902.0,1


In [31]:
# money to be made criteria and conviction-wise
df_portfolio_features.groupby(['Conviction', 'Criteria'])['FTT Amt'].agg(['sum', 'count']).sort_values(by='sum', ascending=False)

sum  count
Conviction Criteria                  
H-SC       XY24      1215239.0      6
           AR         871898.0      5
           XR         788919.0      7
M-SC       XY24       774101.0      6
X-MC       X40        435701.0      7
X-LC       X40        392389.0      5
X-MC       XY24       356441.0      3
H-SC       OX40N      317974.0      4
X-SC       X40N       288509.0      3
M-SC       OX40N      282059.0      5
H-SC       SR         280153.0      2
X-SC       XY24       271839.0      3
X-MC       X40N       242570.0      4
X-LC       XY24       225596.0      2
H-MC       AR         207701.0      2
X-LC       X40N       201509.0      2
H-MC       XY24       188934.0      1
X-MC       XY25       175558.0      2
X-SC       X40        174431.0      2
L-SC       XR         172867.0      2
M-SC       XR         147389.0      2
           AR         122535.0      2
M-LC       XR         117902.0      1
L-SC       OX40N       87501.0      1
X-LC       XY25        72724.0      2
H-SC       MH          72485.0      1
H-LC       AR          67543.0      1
           X200        59225.0      1
L-MC       XR          54083.0      1
M-MC       XY25        49247.0      1
L-LC       XY25        39818.0      1

In [32]:
# notebook execution time

end_time = time.time()
execution_time = round(end_time - start_time, 0)
print(f"Notebook execution time: {execution_time} seconds")

Notebook execution time: 29.0 seconds
